In [2]:
from strawberryfields.apps import data, qchem
from strawberryfields.apps.qchem.vibronic import VibronicTransition
import strawberryfields as sf
import numpy as np
import matplotlib.pyplot as plt
import thewalrus

T = 0  # temperature

In [ ]:
from test import ReadFre,ReadAtoms,ReadNM

In [17]:
ri, m, wi, Li=qchem.read_gamess('gamess/CH2O2_GS.out')
rf, m, wf, Lf=qchem.read_gamess('gamess/CH2O2_ES.out')
wi=wi*0.964
wf=wf*0.964
# Convert to input understood by duschinsky
ri = ri.ravel()
rf = rf.ravel()
m = np.array([[m[i]] * 3 for i in range(len(m))]).flatten()
wi = wi[6:]
wf = wf[6:]
Li = Li[6:].T* m[:, np.newaxis]**0.5
Lf = Lf[6:].T* m[:, np.newaxis]**0.5

In [13]:
(rf-ri)

array([ 0.00692, -0.01336, -0.03982,  0.01659,  0.02473,  0.05189,
       -0.04406, -0.01331,  0.00054,  0.05566,  0.02748,  0.02696,
       -0.03445, -0.02554, -0.03957])

In [20]:
GS="orca/GS/CH2O2GS.hess"
ES="orca/ES/CH2O2ES.hess"
Ai=ReadAtoms(GS)
m=np.repeat(Ai[:,0],3)
ri=(Ai[:,1:4]).flatten()*0.529
Af=ReadAtoms(ES)
rf=(Af[:,1:4]).flatten()*0.529
Li=ReadNM(GS)
Li=Li[:,6:]
Lf=ReadNM(ES)
Lf=Lf[:,6:]
wi=ReadFre(GS)
wi=wi[6:]
wf=ReadFre(ES)
wf=wf[6:]

In [15]:
(rf-ri)

array([ 0.00691772, -0.01336297, -0.03980196,  0.01658095,  0.02473017,
        0.05186606, -0.0440428 , -0.01330839,  0.00054154,  0.05564005,
        0.02747747,  0.02695331, -0.03509591, -0.02553628, -0.03955895])

In [21]:
Ud, delta = qchem.duschinsky(Li, Lf, ri, rf, wf, m)
t, U1, r, U2, alpha = qchem.vibronic.gbs_params(wi, wf, Ud, delta, 0)
r

array([ 0.74963814,  0.48295573,  0.29191516,  0.19995845, -0.01301668,
       -0.38882838, -0.89582196, -1.68706532, -1.81765809])

This is the result of ORCA

In [22]:
from strawberryfields.apps import plot

nr_samples = 100

s = qchem.vibronic.sample(t, U1, r, U2, alpha, nr_samples,loss=0)
e = qchem.vibronic.energies(s, wi, wf)
plot.spectrum(e, xmin=-1000, xmax=8000)